In [1]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
import importlib
import os

from transliteration import data, train, model_one, script, decode

In [2]:
importlib.reload(data)
batch_size = 256
train_dataset = data.make_dataset('../data/tfrecord/train.tfrecord',
                                  from_script='en',
                                  to_script='ja',
                                  batch_size=batch_size)
valid_dataset = data.make_dataset('../data/tfrecord/valid.tfrecord',
                                  from_script='en',
                                  to_script='ja',
                                  batch_size=batch_size)
test_dataset = data.make_dataset('../data/tfrecord/test.tfrecord',
                                 from_script='en',
                                 to_script='ja',
                                 batch_size=batch_size)

In [2]:
batch_size = 128
cmu_train_dataset = data.make_dataset('../data/tfrecord/cmu_train.tfrecord',
                                  from_script='en',
                                  to_script='cmu',
                                  batch_size=batch_size)
cmu_valid_dataset = data.make_dataset('../data/tfrecord/cmu_valid.tfrecord',
                                  from_script='en',
                                  to_script='cmu',
                                  batch_size=batch_size)
cmu_test_dataset = data.make_dataset('../data/tfrecord/cmu_test.tfrecord',
                                 from_script='en',
                                 to_script='cmu',
                                 batch_size=batch_size)

In [15]:
importlib.reload(model_one)
optimizer = tf.train.AdamOptimizer()


def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred)
    return tf.reduce_mean(loss_ * mask)

encoder_config = model_one.Config(lstm_size=120,
                                  embedding_size=60,
                                  attention_size=None,
                                  vocab_size=script.SCRIPTS['en'].vocab_size)
decoder_config = model_one.Config(lstm_size=80,
                                  embedding_size=60,
                                  attention_size=60,
                                  attention='monotonic_bahdanau',
                                  vocab_size=script.SCRIPTS['ja'].vocab_size)
cmu_decoder_config = model_one.Config(lstm_size=80,
                                  embedding_size=60,
                                  attention_size=60,
                                  vocab_size=script.SCRIPTS['cmu'].vocab_size)
encoder = model_one.Encoder(encoder_config)
decoder = model_one.Decoder(decoder_config)
cmu_decoder = model_one.Decoder(cmu_decoder_config)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder,
                                 cmu_decoder=cmu_decoder)

In [ ]:
importlib.reload(train)
best_val_loss = None
for e in range(200):
    loss = train.run_one_epoch(cmu_train_dataset,
                               True,
                               from_script='en',
                               to_script='cmu',
                               encoder=encoder,
                               decoder=cmu_decoder,
                               optimizer=optimizer,
                               loss_function=loss_function)
    valid_loss = train.run_one_epoch(cmu_valid_dataset,
                                     False,
                                     from_script='en',
                                     to_script='cmu',
                                     encoder=encoder,
                                     decoder=cmu_decoder,
                                     loss_function=loss_function)
    if best_val_loss is None or valid_loss < best_val_loss:
        best_val_loss = valid_loss
        checkpoint.save(file_prefix=checkpoint_prefix)
    print("Epoch {}: Train Loss {:.3f}, Valid Loss {:.3f}".format(e, loss, valid_loss))
    print(decode.transliterate(input_strs=['derick'],
                               from_script='en',
                               to_script='cmu',
                               encoder=encoder,
                               decoder=cmu_decoder,
                               k_best=2,
                               decoding_method=decode.beam_search_decode))

In [16]:
importlib.reload(train)
best_val_loss = None
for e in range(200):
    loss = train.run_one_epoch(train_dataset,
                               True,
                               from_script='en',
                               to_script='ja',
                               encoder=encoder,
                               decoder=decoder,
                               train_encoder=True,
                               optimizer=optimizer,
                               loss_function=loss_function)
    valid_loss = train.run_one_epoch(valid_dataset,
                                     False,
                                     from_script='en',
                                     to_script='ja',
                                     encoder=encoder,
                                     decoder=decoder,
                                     loss_function=loss_function)
    if best_val_loss is None or valid_loss < best_val_loss:
        best_val_loss = valid_loss
        checkpoint.save(file_prefix=checkpoint_prefix)
    print("Epoch {}: Train Loss {:.3f}, Valid Loss {:.3f}".format(e, loss, valid_loss))
    print(decode.transliterate(input_strs=['derick'],
                               from_script='en',
                               to_script='ja',
                               encoder=encoder,
                               decoder=decoder,
                               k_best=2,
                               decoding_method=decode.beam_search_decode))

Epoch 0: Train Loss 21.233, Valid Loss 19.356
([['ーー', 'ンー']], array([[-10.15840602, -10.16695213]]))


Epoch 1: Train Loss 18.729, Valid Loss 18.756
([['アルーー', 'アルーーー']], array([[-17.87236977, -20.87293935]]))


Epoch 2: Train Loss 18.207, Valid Loss 18.049
([['アランラン', 'アリンラン']], array([[-20.51289344, -20.73459411]]))


Epoch 3: Train Loss 17.319, Valid Loss 17.099
([['アラント', 'アリント']], array([[-15.59250736, -15.73684573]]))


Epoch 4: Train Loss 15.924, Valid Loss 15.251
([['ディリータート', 'ディアラータート']], array([[-22.44977331, -25.17119241]]))


...

Epoch 26: Train Loss 4.310, Valid Loss 5.323
([['デリック', 'デリックス']], array([[-1.94289263, -3.15559326]]))


Epoch 27: Train Loss 4.176, Valid Loss 5.329
([['デリック', 'デリックス']], array([[-2.01173321, -3.28825078]]))


Epoch 28: Train Loss 4.093, Valid Loss 5.239
([['デリック', 'デリックス']], array([[-1.83368374, -3.17345879]]))


Epoch 29: Train Loss 3.984, Valid Loss 5.249
([['デリック', 'デリックス']], array([[-1.91276402, -3.02227522]]))


KeyboardInterrupt: 

In [37]:
checkpoint.restore(checkpoint_prefix + '-22')

In [34]:
importlib.reload(decode)
decode.transliterate(input_strs=['armor'],
                     from_script='en',
                     to_script='ja',
                     encoder=encoder,
                     decoder=decoder,
                     k_best=2,
                     decoding_method=decode.beam_search_decode)

([['アルモール', 'アーモーロー']], array([[-5.08133715, -5.74089668]]))